# Optimizing Neural Networks with Intel oneAPI and PyTorch Extensions
This notebook demonstrates the process of optimizing neural networks using PyTorch, Intel oneAPI, and Intel Extension for PyTorch (IPEX). 

In [9]:
# Import necessary libraries for computation and deep learning
import numpy as np  # Numerical computing library
import torch  # PyTorch for building and training neural networks
import intel_extension_for_pytorch as ipex  # Intel extension to optimize PyTorch models on Intel hardware
import torchvision  # Useful for computer vision tasks, though not used directly in this notebook

# Print the versions of each library to verify compatibility
print(np.__version__)  # Numpy version
print(torch.__version__)  # PyTorch version
print(ipex.__version__)  # Intel Extension for PyTorch version
print(torchvision.__version__)  # Torchvision version


2.1.2
1.13.1+cpu
1.13.100
0.14.1+cpu


## Importing Necessary Libraries
**Torch**: The core library for PyTorch, enabling tensor operations and GPU support for deep learning.

**torch.nn**: Contains classes and functions for building and managing neural networks, including layers and activation functions. Used for defining the `ImprovedNN` model.

**torch.optim**: Provides optimization algorithms like SGD and **Adam**(used in our case) to update model parameters during training. Used for defining the optimizer in the training process.

**Learning Rate Scheduler**: Dynamically adjusts the learning rate during training to improve convergence. Used for reducing the learning rate when a plateau in loss is detected.

**Scikit-learn Metrics**: Evaluation metrics for classification tasks, helping assess model performance. Used for calculating accuracy, precision, recall, F1-score, AUC, and balanced accuracy.

**train_test_split**: Utility function for splitting the dataset into training and testing subsets. Used in the `data_splitter` function to split the data.

**classification_report**: Generates a report detailing precision, recall, F1-score, and support for each class. Not directly used in the notebook but imported for potential use.

**DataLoader**: Facilitates efficient batching and loading of datasets. Used for creating data loaders for training and testing datasets.

**TensorDataset**: Wraps tensors into a dataset format for compatibility with DataLoader. Used for converting the data into a format compatible with DataLoader.

**NumPy**: A library for numerical computing, providing support for arrays and mathematical functions. Used for various numerical operations and data manipulation.

**Pandas**: A data manipulation library that simplifies data analysis and handling of tabular data. Used for loading and preprocessing the dataset.

**tqdm**: Library for creating progress bars, enhancing user experience during long computations. Not directly used in the notebook but imported for potential use.

**Intel Extension for PyTorch (IPEX)**: This library optimizes the performance of PyTorch models specifically on Intel hardware, providing enhanced execution speed through optimized mathematical operations and efficient memory usage. Used for optimizing the PyTorch models.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import intel_extension_for_pytorch as ipex

## Loading and Preparing Data for Modeling
**Loading the Dataset**: The dataset is read from a specified CSV file using Pandas’ read_csv() function. This file is expected to contain various attributes related to cities, including information on EV stations and other civic amenities.

**Error Handling**: A try-except block is utilized to catch potential FileNotFoundError. This ensures that if the file is not found at the specified path, a clear message is printed to guide the user in troubleshooting.

**Error Handling**: A try-except block is utilized to catch potential FileNotFoundError. This ensures that if the file is not found at the specified path, a clear message is printed to guide the user in troubleshooting.

**Column Filtering**: This line filters the DataFrame to retain only the relevant columns necessary for modeling. The selected features encompass various aspects of urban infrastructure and population, which are likely predictors for the target variable.

**Initial Data Size**: The shape of the DataFrame is printed, giving a quick overview of the number of rows (samples) and columns (features) in the dataset before any preprocessing steps.

**Dropping Missing Values**: This line removes any rows containing missing values (NaN) from the DataFrame. Handling missing values is critical to ensure the dataset is clean and usable for model training, as many algorithms do not accept missing data.

**Post-cleaning Data Size**: After removing rows with missing values, the new shape of the DataFrame is printed. This provides insight into how much data has been retained and how the preprocessing has affected the dataset’s size.


In [ ]:

try:
    data = pd.read_csv("../datasets/processed/all_city_data_with_pop.csv")
    print("Data loaded successfully!")
except FileNotFoundError as e:
    print("File not found. Please check the path:", e)
# Filter columns to be used for modeling
data = data[['geometry', 'city', 'EV_stations', 'parking', 'edges',
             'parking_space', 'civic', 'restaurant', 'park', 'school',
             'node', 'Community_centre', 'place_of_worship', 'university', 'cinema',
             'library', 'commercial', 'retail', 'townhall', 'government',
             'residential', 'population']]

print("Data size:", data.shape)
data = data.dropna()
print("Data size after dropping na:", data.shape)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import intel_extension_for_pytorch as ipex

try:
    data = pd.read_csv("/home/ucce21c3bd21d7702289d768bb49aa80/Training/AI/Introduction_to_PyTorch_24/all_city_data_with_pop.csv")
    print("Data loaded successfully!")
except FileNotFoundError as e:
    print("File not found. Please check the path:", e)
# Filter columns to be used for modeling
data = data[['geometry', 'city', 'EV_stations', 'parking', 'edges',
             'parking_space', 'civic', 'restaurant', 'park', 'school',
             'node', 'Community_centre', 'place_of_worship', 'university', 'cinema',
             'library', 'commercial', 'retail', 'townhall', 'government',
             'residential', 'population']]

print("Data size:", data.shape)
data = data.dropna()
print("Data size after dropping na:", data.shape)

## data_splitter: splits the dataset into training and testing sets based on the input cities
It can handle two scenarios:

	•	If specific train_cities and test_cities are provided, it filters the dataset accordingly.
	•	If no cities are specified, it uses train_test_split to split the data into training and test sets randomly based on the given test_size.

Parameters:

	•	data: The complete dataset.
	•	train_cities: List of cities to be included in the training set (optional).
	•	test_cities: List of cities to be included in the test set (optional).
	•	test_size: Proportion of the data to include in the test split.
	•	random_state: Random seed for reproducibility.

Returns:

	•	X_train: Features of the training set.
	•	X_test: Features of the test set.
	•	y_train: Target labels for the training set.
	•	y_test: Target labels for the test set.

In [ ]:
def data_splitter(data, train_cities=None, test_cities=None, test_size=0.2, random_state=42):
    if train_cities is not None:
        train = data[data['city'].isin(train_cities)]
        test = data[data['city'].isin(test_cities)]

        X_train = train.drop(['city', 'geometry', 'EV_stations'], axis=1)
        y_train = train['EV_stations'].astype(int)
        y_train = y_train.apply(lambda x: 1 if x > 0 else 0)

        X_test = test.drop(['city', 'geometry', 'EV_stations'], axis=1)
        y_test = test['EV_stations'].astype(int)
        y_test = y_test.apply(lambda x: 1 if x > 0 else 0)
    else:
        X = data.drop(['city', 'geometry', 'EV_stations'], axis=1)
        y = data['EV_stations']
        y = y.apply(lambda x: 1 if x > 0 else 0)
        X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = data_splitter(data)

This block converts the X_train, X_test, y_train, and y_test datasets from Pandas DataFrames into PyTorch tensors. Tensors are the core data structures in PyTorch used to train the neural network.

	•	The .unsqueeze(1) operation reshapes the labels (y_train and y_test) to have a second dimension, making it compatible with the neural network’s output shape.

In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

## ImprovedNN :A Neural Network Model with Batch Normalization and Dropout

This class defines a custom neural network model. It inherits from nn.Module, the base class for all neural network modules in PyTorch.

Architecture:

	•	fc1: The first fully connected layer with 256 units.
	•	bn1: Batch normalization applied after the first layer.
	•	dropout1: Dropout layer to prevent overfitting by randomly deactivating neurons during training.
	•	fc2: Second fully connected layer with 128 units.
	•	bn2: Batch normalization applied after the second layer.
	•	dropout2: Dropout layer.
	•	fc3: Third fully connected layer with 64 units.
	•	fc4: Output layer with a single unit using sigmoid activation to produce a probability output for binary classification.

The activation function used between layers is ReLU for non-linearity, and sigmoid is applied at the output to restrict predictions between 0 and 1.

In [ ]:
class ImprovedNN(nn.Module):
    def __init__(self):
        super(ImprovedNN, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = torch.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = torch.relu(self.fc3(x))
        x = torch.sigmoid(self.fc4(x))
        return x

## Early Stopping Mechanism: Technique to prevent overfitting during training
Early stopping is a technique to prevent overfitting during training. If the model’s loss does not improve after a certain number of epochs, training is stopped.

Attributes:

	•	patience: The number of epochs to wait for an improvement in loss before stopping training.
	•	min_delta: Minimum change in the monitored loss to qualify as an improvement.

Functionality:

	•	step(loss): Monitors the loss at each epoch and stops the training if it doesn’t improve after patience epochs.


In [ ]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.01):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0

    def step(self, loss):
        if self.best_loss is None:
            self.best_loss = loss
        elif self.best_loss - loss > self.min_delta:
            self.best_loss = loss
            self.counter = 0
        else:
            self.counter += 1

        if self.counter >= self.patience:
            return True
        return False

## Weight Initialization:Xavier (Glorot) initialization
This function applies Xavier (Glorot) initialization to the weights of the linear layers. Proper weight initialization can help with better convergence during training.

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)


## Training Function: Handles the training process of the neural network
This function handles the training process of the neural network.

Parameters:

	•	model: The neural network model to be trained.
	•	optimizer: The optimization algorithm (Adam in this case) that updates the model’s weights.
	•	criterion: The loss function (Binary Cross-Entropy).
	•	scheduler: Learning rate scheduler to adjust the learning rate based on the loss.
	•	early_stopping: Early stopping instance to monitor loss and stop training early if necessary.
	•	train_loader: DataLoader for the training data.
	•	val_loader: DataLoader for the validation data.
	•	epochs: Number of training epochs.
	•	device: The device to run the model on (CPU or GPU).

Process:

	•	The function loops through each epoch, performs a forward pass, calculates the loss, performs backpropagation, and updates the weights.
	•	It also implements gradient clipping to prevent exploding gradients.
	•	The scheduler adjusts the learning rate based on the loss at the end of each epoch.
	•	Early stopping is checked, and if triggered, training stops early.

In [ ]:
# Training Function
def train_model(model, optimizer, criterion, scheduler, early_stopping, train_loader, val_loader, epochs=50, device='cpu'):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = model(inputs.float())
            loss = criterion(outputs, labels.unsqueeze(1).float())

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)

            optimizer.step()

            running_loss += loss.item()

        # Scheduler step
        scheduler.step(running_loss)

        # Early Stopping check
        if early_stopping.step(running_loss):
            print(f"Early stopping at epoch {epoch}")
            break

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss:.4f}")

    return model

## Model Evaluation Function:Evaluates the trained model on the test dataset &&  Optimizing Performance with Intel Extension for PyTorch (IPEX)
**Function Definition**: This function evaluates the performance of a given neural network model on a test dataset.
	•	Parameters:
	•	model: The trained PyTorch model to be evaluated.
	•	test_loader: A DataLoader object that provides the test dataset in batches.
	•	device: The device to run the model on ('cpu' or 'cuda'). Defaults to 'cpu'.

**Set Model to Evaluation Mode**: This method sets the model to evaluation mode, which disables dropout and batch normalization, ensuring consistent outputs.

**Model Optimization**: Uses **Intel Extension for PyTorch (IPEX)** to **optimize the model** for better performance on Intel hardware.

**Return True and Predicted Labels**: Converts both lists of true labels and predictions into NumPy arrays and returns them, allowing for further analysis of the model’s performance.

In [ ]:
def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    model=ipex.optimize(model)
    y_true = []
    y_pred = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs.float())
            predictions = (outputs > 0.5).float().cpu().numpy()
            y_pred.extend(predictions)
            y_true.extend(labels.cpu().numpy())

    return np.array(y_true), np.array(y_pred)

## run_experiment:Sets up and runs the entire experiment
This is the main function that sets up and runs the entire experiment.

Process:

	•	The data is first converted into PyTorch tensors and wrapped into DataLoader objects for batching.
	•	A neural network model (ImprovedNN) is instantiated and initialized with Xavier initialization.
	•	The optimizer, loss function, learning rate scheduler, and early stopping mechanism are defined.
	•	The model is trained using the train_model function.
	•	After training, the model is evaluated using the evaluate_model function.

Evaluation Metrics:

	•	Accuracy: The proportion of correct predictions.
	•	Precision: The number of true positives divided by the number of predicted positives.
	•	Recall: The number of true positives divided by the number of actual positives.
	•	F1-score: The harmonic mean of precision and recall.
	•	AUC: Area under the ROC curve, a performance measure for binary classification.
	•	Balanced Accuracy: Accuracy adjusted for imbalanced datasets.

In [ ]:
def run_experiment(X_train, X_test, y_train, y_test, batch_size=32, epochs=50):
    # Convert Pandas DataFrames to NumPy arrays first
    X_train_np = X_train.values
    X_test_np = X_test.values
    y_train_np = y_train.values
    y_test_np = y_test.values

    # Convert data to PyTorch tensors and create DataLoader
    train_dataset = TensorDataset(torch.tensor(X_train_np), torch.tensor(y_train_np))
    test_dataset = TensorDataset(torch.tensor(X_test_np), torch.tensor(y_test_np))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    # Define the device (use GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Initialize model, optimizer, and loss function
    model = ImprovedNN().to(device)
    model.apply(weights_init)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    # Define Learning Rate Scheduler and Early Stopping
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, verbose=True)
    early_stopping = EarlyStopping(patience=50)

    # Train the model
    model = train_model(model, optimizer, criterion, scheduler, early_stopping, train_loader, test_loader, epochs=epochs, device=device)

    # Evaluate the model
    y_true, y_pred = evaluate_model(model, test_loader, device=device)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')
    auc = roc_auc_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

    # Return metrics and the trained model
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "AUC": auc,
        "Balanced Accuracy": balanced_accuracy
    }, model


metrics, trained_model = run_experiment(X_train, X_test, y_train, y_test)
print(metrics)

Data loaded successfully!
Data size: (10824, 22)
Data size after dropping na: (10129, 22)
Epoch 1/50, Loss: 80.4817
Epoch 2/50, Loss: 68.2528
Epoch 3/50, Loss: 67.3190
Epoch 4/50, Loss: 66.2678
Epoch 5/50, Loss: 65.5271
Epoch 6/50, Loss: 64.1401
Epoch 7/50, Loss: 64.5227
Epoch 8/50, Loss: 63.6566
Epoch 10/50, Loss: 63.7573
Epoch 11/50, Loss: 62.5218
Epoch 12/50, Loss: 63.7462
Epoch 13/50, Loss: 62.7203
Epoch 14/50, Loss: 63.1687
Epoch 16/50, Loss: 62.6809
Epoch 00017: reducing learning rate of group 0 to 1.0000e-04.
Epoch 17/50, Loss: 62.8853
Epoch 18/50, Loss: 62.5329
Epoch 19/50, Loss: 62.2598
Epoch 20/50, Loss: 62.0549
Epoch 21/50, Loss: 61.8264
Epoch 22/50, Loss: 62.0168
Epoch 23/50, Loss: 61.5555
Epoch 24/50, Loss: 61.6650
Epoch 25/50, Loss: 61.9277
Epoch 26/50, Loss: 60.9742
Epoch 27/50, Loss: 61.2287
Epoch 28/50, Loss: 61.3921
Epoch 29/50, Loss: 61.8382
Epoch 30/50, Loss: 62.0283
Epoch 31/50, Loss: 61.1148
Epoch 00032: reducing learning rate of group 0 to 1.0000e-05.
Epoch 32/50, Loss: 61.9968
Epoch 33/50, Loss: 61.4239
Epoch 35/50, Loss: 61.5319
Epoch 36/50, Loss: 61.1627
Epoch 37/50, Loss: 61.4312
Epoch 00038: reducing learning rate of group 0 to 1.0000e-06.
Epoch 38/50, Loss: 61.4890
Epoch 39/50, Loss: 61.9879
Epoch 40/50, Loss: 60.0656
Epoch 41/50, Loss: 61.6155
Epoch 42/50, Loss: 60.9621
Epoch 43/50, Loss: 60.5539
Epoch 44/50, Loss: 61.1476
Epoch 45/50, Loss: 61.1083
Epoch 47/50, Loss: 61.4583
Epoch 48/50, Loss: 61.3163
Epoch 49/50, Loss: 60.7892
Epoch 50/50, Loss: 61.1086
{'Accuracy': 0.903751233958539, 'Precision': 0.8501829109833214, 'Recall': 0.6226040500186636, 'F1-score': 0.6663246044113362, 'AUC': 0.6226040500186637, 'Balanced Accuracy': 0.6226040500186636}
